In [1]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import itertools
import scipy.io as sio
from sklearn.model_selection import KFold
import csv
from scipy.linalg import inv
import random
import os
import sys

In [14]:
subs = []

data_2 = np.zeros((20, 540, 64*5))
data_7 = np.zeros((20, 630, 64*5))

# load data (20 subjects, just for example)

In [ ]:
# labels: 2 class
group_2block_normal2 = [0]*270 + [1]*270
group_2block_normal2 = np.array(group_2block_normal2)
normal_label_per_subject2 = group_2block_normal2
print("normal_label_per_subject2:", normal_label_per_subject2.shape)

group_2block_11_ima2 = normal_label_per_subject2
group_2block_11_vid2 = normal_label_per_subject2

mix = np.concatenate((group_2block_11_ima2, group_2block_11_vid2), axis=0)
print("mix:", mix.shape)

In [ ]:
# labels: 7 class
group_block_normal7 = [0]*90 + [1]*90 + [2]*90 + [3]*90 + [4]*90 + [5]*90 + [6]*90
group_block_normal7 = np.array(group_block_normal7)
normal_label_per_subject7 = group_block_normal7
print("normal_label_per_subject7:", normal_label_per_subject7.shape)

mix = np.concatenate((group_block_normal7, group_block_normal7), axis=0)
print("mix:", mix.shape)

In [ ]:
# intra-subject

train_acc = []
test_acc = []

for sub_idx in range(20):
    test_list_all = []
    train_list_all = []
    for fold in range(3):

        test_list_fold = []
        train_list_fold = []
        for block_idx in range(6):
            test_list = list(range(block_idx*90 + fold*30, block_idx*90 + fold*30 + 30))
            test_list_fold.append(test_list)
            # train_list is the rest of the blocks
            train_list = list(set(range(block_idx*90, block_idx*90+90))-set(test_list))
            train_list_fold.append(train_list)
    
        test_list_all.append(test_list_fold)
        train_list_all.append(train_list_fold)

    test_list_all = np.array(test_list_all)
    train_list_all = np.array(train_list_all)

for sub_idx in range(20):
    train_acc_fold = []
    test_acc_fold = []   
    for fold in range(3):      
        test_list = test_list_all[fold]
        train_list = train_list_all[fold]
        test_list = test_list.flatten()
        train_list = train_list.flatten()

        train_data = data_2[sub_idx, train_list, :]
        train_label = normal_label_per_subject2[train_list]

        test_data = data_2[sub_idx, test_list, :]
        test_label = normal_label_per_subject2[test_list]

        C_values = np.logspace(-3, 3,num=7)
        test_acc_perfold = 0
        train_acc_perfold = 0
        for C in C_values:
            clf = LinearSVC(random_state=5, C=C)
            clf.fit(train_data, train_label)
            # print("C:", C, "train acc:", clf.score(train_data, train_label), "test acc:", clf.score(test_data, test_label))
            train_acc_C = clf.score(train_data, train_label)
            test_acc_C = clf.score(test_data, test_label)
            print("sub:",subs[sub_idx],"C:", C, "train acc:", train_acc_C, "test acc:", test_acc_C)
            if test_acc_C > test_acc_perfold:
                test_acc_perfold = test_acc_C
                train_acc_perfold = train_acc_C
        train_acc_fold.append(train_acc_perfold)
        test_acc_fold.append(test_acc_perfold)
        # print("train acc for sub", sub_idx, ":", train_acc_perfold)
        # print("test acc for sub", sub_idx, ":", test_acc_perfold)
    train_acc.append(train_acc_fold)
    test_acc.append(test_acc_fold)
    # print("train acc for sub", sub_idx, ":", train_acc_fold)
    # print("test acc for sub", sub_idx, ":", test_acc_fold)
train_acc = np.array(train_acc)
test_acc = np.array(test_acc)
train_acc = np.mean(train_acc, axis=0)
test_acc = np.mean(test_acc, axis=0)
test_std = np.std(test_acc)
print("train_acc:", train_acc)
print("test_acc:", test_acc)
print("test_std:", test_std)

In [ ]:
# inter_subject
# leave-one-out

# train_acc = []
# test_acc = []

# for i in range(20):
#     print(f"-------------sub:{i}----------------")
#     test_data = data_7[i]
#     train_data = np.delete(data_7, i, axis=0)
#     train_data = train_data.reshape(-1, 320)
#     test_label = normal_label_per_subject7
#     train_label = np.tile(normal_label_per_subject7, 19)
#     # random.shuffle(train_label)
#     print(f"train_data:{train_data.shape}, test_data:{test_data.shape}")
#     print(f"train_label:{train_label.shape}, test_label:{test_label.shape}")

#     C_values = np.logspace(-3, 3, num=7)
#     test_acc_persub = 0
#     train_acc_persub = 0

#     for C in C_values:
#         clf = LinearSVC(C=C, max_iter=10000)
#         clf.fit(train_data, train_label)
#         train_acc_persub = max(train_acc_persub, clf.score(train_data, train_label))
#         test_acc_persub = max(test_acc_persub, clf.score(test_data, test_label))
#         # print(f"C:{C}, train_acc:{train_acc_persub}, test_acc:{test_acc_persub}")
#     train_acc.append(train_acc_persub)
#     test_acc.append(test_acc_persub)
#     print(f"sub:{i},train_acc:{train_acc_persub}, test_acc:{test_acc_persub}")

# train_acc = np.array(train_acc)
# test_acc = np.array(test_acc)

# train_std = np.std(train_acc)
# test_std = np.std(test_acc)

# train_acc = np.mean(train_acc)
# test_acc = np.mean(test_acc)

# print(f"train_acc:{train_acc}, train_std:{train_std}")
# print(f"test_acc:{test_acc}, test_std:{test_std}")